## Process Description

![Dietician](Assets/Simpy_clinic.png)

Simulate a process of dietary consultation. People visit the clinic, register at the front desk and then consult the dietician.

1. **Entity**: People arriving for dietary consultation
2. **Generator**: Generate arrivals for dietary consultation 
3. **Inter-Arrrival Time**: An exponential distribution sampled for patient arrivals  
4. **Activity**: Registration followed by consultation with dietician
5. **Activity Time**: Exponential distributions
6. **Resources**: Receptionist, dietician
7. **Queues**: People waiting for registration after arrivel and for consultation after registration
8. **Sink**: Exit after consultation

Based on: https://youtu.be/jXDjrWKcu6w

### 1. Import packages

Aside from **simpy**, we need **random** to sample from random distributions.

In [30]:
import simpy
import random

### 2. Generate Arrivals

Define the generator for arrivals. This puts patients in the queue for consultation.

**Args**:
- env (Simpy environment) required
- mean_IAT (int) mean Inter-Arrival Time for generating arrivals
- meanCT2register (int) mean Cycle Time for registration
- meanCT2consult (int) mean Cycle Time for consultation
- mean_CT (int) mean consultation time
- receptionist (Simpy resource) resource
- dietician (Simpy resource) resource

**Notes**:
- Has a forever-while loop in which it:
	- Creates a patient ID and sends the patient onward on their journey, invoking the function that generates consultations with the patient's ID.
	- Has one ```yield``` statement to timeout for sampled Inter-Arrival Time, after which it increments patient ID. The ```yield``` with timeout will await the action (i.e. completion of timer), freezing until then and next resuming where it left off.

In [31]:
def generate_arrivals(env, mean_IAT, meanCT2register, meanCT2consult, receptionist, dietician):
    # Create patient zero
    patient_ID = 0

    while True:
        # Send patient on the way
        action = generate_action(env, meanCT2register, meanCT2consult, receptionist, dietician, patient_ID)
        env.process(action)

        # Calculate the inter-arrival time before next patient arrives
        delta = random.expovariate(1.0 / mean_IAT)
        yield env.timeout(delta)

        # Next patient
        patient_ID += 1

### 3. Generate Action

Define the generator for registration and consulation.

**Args**:
- env (Simpy environment) required
- mean_CT2register (int) mean registration time
- mean_CT2consult (int) mean consultation time
- receptionist (Simpy resource) resource
- dietician (Simpy resource) resource
- patient_ID (int) patient ID from the function that generates arrivals

**Notes**:
- Has a with statement for registration to request a receptionist with which to timeout for sampled registration time. Will timeout until resource is available.
- Has a with statement for consultation to request a dietician with which to timeout for sampled consultation time. Will timeout until resource is available.
- Notes timestamps at: entered queue for registration, started registration, entered queue for consultation, started consultation, exited process.  

In [32]:
def generate_action(env, mean_CT2register, mean_CT2consult, receptionist, dietician, patient_ID):
    # timestamp queued for registration
    queued4registration = env.now

    # request receptionist
    with receptionist.request() as req:
        yield req

        # timestamp started registration
        startedRegistration = env.now
        print("{} started registration at {:.2f} after waiting {:.2f}".format(patient_ID, startedRegistration, 
                startedRegistration-queued4registration))

        # do! registration        
        delta_registration = random.expovariate(1.0 / mean_CT2register) 
        yield env.timeout(delta_registration)

    # timestamp queued for consultation
    queued4consultation = env.now

    # request dietician
    with dietician.request() as req:
        yield req
        
        # timestamp started consultation
        startedConsultation = env.now
        print("{} started consultation at {:.2f} after waiting {:.2f}".format(patient_ID, startedConsultation, 
                startedConsultation-queued4consultation))

        # do consultation
        delta_consultation = random.expovariate(1.0 / mean_CT2consult)
        yield env.timeout(delta_consultation)

    # timestamp exited
    exited = env.now
    print("{} left at {:.2f}".format(patient_ID, exited))

*Question*: The two ```with``` statements in ```generate_action()``` are at the same level. If one were nested within the other, how does the meaning change?

### 4. Set Up & Run Simulation

Define simulation variables as follows:
- Create an instance of env
- Initialize resources
- Define key parameters (for distributions)

In [33]:
# Set up the simulation environment
env = simpy.Environment()

# Set up the resources
receptionist = simpy.Resource(env, 1)
dietician = simpy.Resource(env, 1)

# Configure simulation parameters
mean_IAT = 5 
mean_CT2register = 6
mean_CT2consult = 8

In [34]:
env.process(generate_arrivals(env, mean_IAT, mean_CT2register, mean_CT2consult, receptionist, dietician))
env.run(until=120)

0 started registration at 0.00 after waiting 0.00
0 started consultation at 11.25 after waiting 0.00
1 started registration at 11.25 after waiting 8.79
0 left at 14.22
1 started consultation at 26.47 after waiting 0.00
2 started registration at 26.47 after waiting 6.13
1 left at 33.88
2 started consultation at 34.21 after waiting 0.00
3 started registration at 34.21 after waiting 9.54
4 started registration at 41.19 after waiting 13.35
5 started registration at 41.26 after waiting 3.02
2 left at 41.83
3 started consultation at 41.83 after waiting 0.65
3 left at 44.62
4 started consultation at 44.62 after waiting 3.36
4 left at 45.06
5 started consultation at 50.52 after waiting 0.00
6 started registration at 50.52 after waiting 5.89
5 left at 58.35
6 started consultation at 59.89 after waiting 0.00
7 started registration at 59.89 after waiting 15.13
8 started registration at 63.13 after waiting 15.62
9 started registration at 69.38 after waiting 20.12
10 started registration at 71.68 a

## Part B: Archive Data

So far we have printed out results. Now, let us save them using arrays. Define global arrays for saving timestamps. Then proceed to persist information to file.

We want to know how efficient is our process. Efficiency means minimizing wster. From a patient's vantage point, waste is time spent in queuing. We shall persist information about time spent in queuing and the lead time from arrival to exit.


### 1. Generate Arrivals

Modify the generator for arrivals to use a new generator function to process arrivals.

In [35]:
def generate_arrivals2(env, mean_IAT, meanCT2register, meanCT2consult, receptionist, dietician):
    # Create patient zero
    patient_ID = 0

    while True:
        # Send patient on the way
        action = generate_action2(env, meanCT2register, meanCT2consult, receptionist, dietician, patient_ID)
        env.process(action)

        # Calculate the inter-arrival time before next patient arrives
        delta = random.expovariate(1.0 / mean_IAT)
        yield env.timeout(delta)

        # Next patient
        patient_ID += 1

### 2. Generate Action

Modify the function to use globals to persist information.

In [36]:
def generate_action2(env, mean_CT2register, mean_CT2consult, receptionist, dietician, patient_ID):
    # globals
    global queued4registration
    global queued4consultation
    global leadTime
    
    # timestamp queued for registration
    arrived = env.now

    # request receptionist
    with receptionist.request() as req:
        yield req

        # timestamp started registration
        startedRegistration = env.now
        print("{} started registration at {:.2f} after waiting {:.2f}".format(patient_ID, startedRegistration, 
                startedRegistration-arrived))
        queued4registration.append(startedRegistration - arrived)

        # do! registration        
        delta_registration = random.expovariate(1.0 / mean_CT2register) 
        yield env.timeout(delta_registration)

    # timestamp queued for consultation
    enteredQueue4consultation = env.now

    # request dietician
    with dietician.request() as req:
        yield req
        
        # timestamp started consultation
        startedConsultation = env.now
        print("{} started consultation at {:.2f} after waiting {:.2f}".format(patient_ID, startedConsultation, 
                startedConsultation-enteredQueue4consultation))
        queued4consultation.append(startedConsultation-enteredQueue4consultation)

        # do consultation
        delta_consultation = random.expovariate(1.0 / mean_CT2consult)
        yield env.timeout(delta_consultation)

    # timestamp exited
    exited = env.now
    print("{} left at {:.2f}".format(patient_ID, exited))
    leadTime.append(exited-arrived)

### 3. Set Up & Run Simulation

Initialize the globals.

In [40]:
# Set up the simulation environment
env = simpy.Environment()

# Set up the resources
receptionist = simpy.Resource(env, 1)
dietician = simpy.Resource(env, 2)

# Configure simulation parameters
mean_IAT = 5 
mean_CT2register = 6
mean_CT2consult = 8

# Globals
queued4registration = []
queued4consultation = []
leadTime = []

In [41]:
# Make it so!
env.process(generate_arrivals2(env, mean_IAT, mean_CT2register, mean_CT2consult, receptionist, dietician))
env.run(until=240)

0 started registration at 0.00 after waiting 0.00
0 started consultation at 7.02 after waiting 0.00
1 started registration at 7.02 after waiting 5.46
1 started consultation at 7.32 after waiting 0.00
2 started registration at 7.32 after waiting 5.54
1 left at 9.22
2 started consultation at 9.22 after waiting 0.58
0 left at 9.52
3 started registration at 10.23 after waiting 0.00
2 left at 16.33
3 started consultation at 20.44 after waiting 0.00
4 started registration at 20.44 after waiting 9.05
4 started consultation at 23.43 after waiting 0.00
5 started registration at 23.43 after waiting 4.92
3 left at 23.48
4 left at 33.00
5 started consultation at 34.22 after waiting 0.00
6 started registration at 36.95 after waiting 0.00
5 left at 39.06
6 started consultation at 39.55 after waiting 0.00
7 started registration at 42.20 after waiting 0.00
7 started consultation at 55.05 after waiting 0.00
8 started registration at 55.05 after waiting 8.40
6 left at 56.17
7 left at 57.04
8 started con

In [43]:
# Persisted info.

leadTime

[7.665743013518509,
 9.522000073316137,
 14.543975037127899,
 13.25355467636421,
 21.60890761268535,
 20.555296364027182,
 19.228368654738873,
 14.841809741713213,
 23.745707167737542,
 21.253539466335006,
 36.42964178353322,
 23.141372642735348,
 43.36784732020676,
 25.353844654041026,
 26.37189054665052,
 26.856787094649732,
 37.71764195001195,
 32.313192465103455,
 37.63251994768372,
 27.68129771817813,
 26.324989480079154,
 42.19091610068821,
 39.6515944442893,
 49.793177711298725,
 59.66918380658237,
 53.66845996699047,
 58.06858581227068,
 53.47504637248474,
 46.95407099439913,
 72.36604819798737,
 48.93325771832542,
 48.89910788736043,
 39.07620971194615,
 38.65641006258625,
 42.25980782377164,
 49.23701902062763,
 57.074606000902804,
 52.72509888572685,
 55.248543795649994,
 63.39292882132247]